In [ ]:
import torch

from torch import nn

from models.tpr_block_vit_hf import TPViT

In [ ]:
# Register forward hook on last qkvr layer.

activation = {}

def get_activation(name):
    def hook(module, module_in, module_out):
        activation[name] = module_out.detach().numpy()
    return hook

# my ori tp-vit
#model.model.model[1][11].encoder[0].fn[0].qkvr.register_forward_hook(get_activation('qkvr'))

# lucidrains tp-vit
#model.model.transformer.layers[11][0].fn.to_qkvr.register_forward_hook(get_activation('qkvr'))

# t2t-vit-t-14
model.model.blocks[13].attn.qkvr.register_forward_hook(get_activation('qkvr'))

In [ ]:
# Set up synset to class label conversion
label_dict = {}

with open('synset_to_class.txt') as f:
    for line in f:
        key, value = line.strip().split(':')
        key = int(key)
        label_dict[key] = value

In [ ]:
# Function to return either q, k, v, or r vectors for a particular/all heads.

def get_qkvr_vector(image, activation, vector='r', all_heads=True, head_num=0):
    probs = model(image)
    
    if all_heads:
        qkvr = rearrange(activation['qkvr'], 'b n (h d qkvr) -> (qkvr) b (h n) d', h=8, qkvr=4)
    else:
        qkvr = rearrange(activation['qkvr'], 'b n (h d qkvr) -> (qkvr) (b h) n d', h=8, qkvr=4)
        
    idx = 'qkvr'.index(vector)
    z = qkvr[idx]
    
    if all_heads:
        z = z[0]
    else:
        z = z[head_num] # Select single head.
        
    return probs, z